# Get cells within R radius of contacts

author: steeve.laquitaine@epfl.ch

TODO:
- Sanity check

In [10]:
# SET PACKAGES
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import h5py
import bluepy as bp
pd.set_option('display.max_colwidth', None)


# SET PATHS
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting"

os.chdir(PROJ_PATH)
from src.nodes.utils import get_config


# SET PARAMETERS
EXPERIMENT = "supp/silico_reyes"       # specifies the experiment from conf/
SIMULATION_DATE = "2023_01_13"          # specifies the run (date)
CONTACT_IDS = np.arange(127, 127+32, 1) # selected contact ids from Neuropixels-384 template for our probe
nsites = 128


# SETUP CONFIG
data_conf, param_conf = get_config(EXPERIMENT, SIMULATION_DATE).values()


# SET OUTPUT FIGURE PATHS
FIG_PATH = data_conf["figures"]["silico"]["probe_location"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-04-28 16:41:03,410 - root - utils.py - get_config - INFO - Reading experiment config.
2023-04-28 16:41:03,476 - root - utils.py - get_config - INFO - Reading experiment config. - done


In [11]:
def load_locations_from_weights_reyes(weightspath: str, nsites: int):
    """Load the channel site locations from the weights file

    Args:
        weightspath (str): _description_
        nsites (int): _description_

    Returns:
        _type_: _description_
    """
    # loading weights file with channel locations
    f = h5py.File(weightspath, "r")

    locationstmp = np.array([0, 0, 0])
    for i in range(nsites):
        temp = np.array(f["electrodes"]["reyespuerta_" + str(i)]["location"])
        locationstmp = np.c_[locationstmp, temp]
    locations = locationstmp.T[1:]
    return locations


def load_locations_from_weights(data_conf: dict, nsites: int):
    """load channel locations from the channel weight file
    of this experiment

    Args:
        data_conf (dict): _description_
        nsites (int): _description_

    Returns:
        _type_: _description_
    """

    # get weight file path
    WEIGHT_PATH = data_conf["dataeng"]["chunks"]["input"]["weight"]

    # select experiment config
    if data_conf["exp"] == "supp/silico_reyes":
        return load_locations_from_weights_reyes(WEIGHT_PATH, nsites)

In [20]:
# load channel locations
locations = load_locations_from_weights(data_conf, nsites)

In [13]:
# read simulaton
simpath = "/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/raw/reyes_probe_lfp_4sec_2023_01_11/1/"
sim = bp.Simulation(simpath + "/BlueConfig")

2023-04-28 16:42:17,474 - bluepy_configfile.configfile - configfile.py - verify_type - WARNING - Invalid absolute path: /gpfs/bbp.cscs.ch/project/proj45/scratch/S1full/METypes
2023-04-28 16:42:17,483 - bluepy_configfile.configfile - configfile.py - validate - WARNING - Have keys ({'KeepModelData', 'LFPWeightsPath'}) that are not defined for section: Run


In [28]:
RADIUS = 50


def find_close_gids(cells, probepos, radius: int):
    cpos = cells[list("xyz")]
    tiled = np.tile(probepos, (cpos.shape[0], 1))
    dist = np.linalg.norm(cpos - tiled, axis=1)
    return cells[dist < radius]


# find near-probe cells within RADIUS microns to one contact
cells = sim.circuit.cells.get({"$target": "hex0"})
all_close_gids = []
for location in locations:
    close_gids_df = find_close_gids(cells, location, RADIUS)
    close_gids = list(close_gids_df.index)
    all_close_gids = all_close_gids + close_gids
    all_close_gids = list(set(all_close_gids))

In [29]:
all_close_gids

[]